In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d anshulm257/rice-disease-dataset

Dataset URL: https://www.kaggle.com/datasets/anshulm257/rice-disease-dataset
License(s): unknown
 99% 0.98G/0.99G [00:04<00:00, 247MB/s]
100% 0.99G/0.99G [00:04<00:00, 228MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/rice-disease-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
import shutil
import os

train_dir = "/content/Rice_Leaf_AUG"

for subset_dir in [train_dir]:
    healthy_path = os.path.join(subset_dir, "Healthy Rice Leaf")
    if os.path.exists(healthy_path):
        shutil.rmtree(healthy_path)
        print(f"✅ Removed: {healthy_path}")
    else:
        print(f"⚠️ Skipped (not found): {healthy_path}")


✅ Removed: /content/Rice_Leaf_AUG/Healthy Rice Leaf


In [ ]:
import shutil
import os

train_dir = "/content/output/train"
test_dir="/content/output/val"

for subset_dir in [train_dir, test_dir]:
    healthy_path = os.path.join(subset_dir, "Brown Spot")
    if os.path.exists(healthy_path):
        shutil.rmtree(healthy_path)
        print(f"✅ Removed: {healthy_path}")
    else:
        print(f"⚠️ Skipped (not found): {healthy_path}")


✅ Removed: /content/output/train/Brown Spot
✅ Removed: /content/output/val/Brown Spot


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio("/content/Rice_Leaf_AUG", output="output", seed=1337, ratio=(.8, 0.2))

Copying files: 3176 files [00:06, 457.72 files/s]


In [ ]:
def preprocess(image,label):
  image=tf.image.resize(image,(224,224))
  return tf.cast(image,tf.float32)/255,label

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_data=train_gen.flow_from_directory(
    directory="/content/output/train",
    target_size=(256,256),
    class_mode="categorical",
    batch_size=32
)
test_data=test_gen.flow_from_directory(
    directory="/content/output/val",
    target_size=(256,256),
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

Found 2022 images belonging to 4 classes.
Found 508 images belonging to 4 classes.


In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    directory="/content/train_subset",
    labels="inferred",
    label_mode="int",
    shuffle=True,
    batch_size=32,
    image_size=(224,224)
)
test_ds=tf.keras.utils.image_dataset_from_directory(
    directory="/content/test_subset",
    labels="inferred",
    label_mode="int",
    shuffle=True,
    batch_size=32,
    image_size=(224,224)
)
train_ds=train_ds.map(preprocess)
test_ds=test_ds.map(preprocess)

Found 12052 files belonging to 8 classes.
Found 2651 files belonging to 8 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

l2_reg = 3.377987006901106e-05
dropout_rate = 0.5973131140901382
learning_rate = 0.0003348462009439816

model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(256,256,3), kernel_regularizer=l2(l2_reg)),
    Activation('relu'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(l2_reg)),
    Activation('relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), padding='same'),
    Activation('relu'),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), padding='same'),
    Activation('relu'),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), padding='same'),
    Activation('relu'),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(dropout_rate),
    Dense(4, activation='softmax')
])

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_116 (Conv2D)             │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_131 (Activation)     │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_100               │ (None, 128, 128, 32)   │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_117 (Conv2D)             │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_132 (Activation)     │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_101               │ (None, 64, 64, 64)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_118 (Conv2D)             │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_133 (Activation)     │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_102               │ (None, 32, 32, 128)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_119 (Conv2D)             │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_134 (Activation)     │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_103               │ (None, 16, 16, 256)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_120 (Conv2D)             │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_135 (Activation)     │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_104               │ (None, 8, 8, 256)      │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,076,292 (11.74 MB)

 Trainable params: 3,076,292 (11.74 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_data,epochs=20,validation_data=test_data)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 53s 740ms/step - accuracy: 0.2948 - loss: 1.3701 - val_accuracy: 0.5394 - val_loss: 1.1960
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 689ms/step - accuracy: 0.4512 - loss: 1.2305 - val_accuracy: 0.5512 - val_loss: 1.1654
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 673ms/step - accuracy: 0.5157 - loss: 1.1469 - val_accuracy: 0.6122 - val_loss: 0.9766
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 687ms/step - accuracy: 0.5767 - loss: 1.0459 - val_accuracy: 0.5886 - val_loss: 1.0614
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 673ms/step - accuracy: 0.6180 - loss: 0.9787 - val_accuracy: 0.6122 - val_loss: 0.9598
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 673ms/step - accuracy: 0.6216 - loss: 0.9404 - val_accuracy: 0.6594 - val_loss: 0.8683
Epoch 7/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 677ms/step - accuracy: 0.6139 - loss: 0.9247 - val_accuracy: 0.6693 - val_loss: 0.8690
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 678ms/step - accuracy: 0.6324 - loss: 0.9419 - val_accu

In [ ]:
import numpy as np
y_pred=model.predict(test_data)
y_pred=np.argmax(y_pred,axis=1)
from sklearn.metrics import classification_report,accuracy_score
print(accuracy_score(test_data.classes,y_pred))
print(classification_report(test_data.classes,y_pred))

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 183ms/step
0.8169291338582677
              precision    recall  f1-score   support

           0       0.73      0.82      0.77       128
           1       0.86      0.79      0.82       127
           2       0.81      0.79      0.80       126
           3       0.87      0.87      0.87       127

    accuracy                           0.82       508
   macro avg       0.82      0.82      0.82       508
weighted avg       0.82      0.82      0.82       508



In [ ]:

from rembg import remove
from PIL import Image

input_path = "/content/output/train/Leaf Blast/IMG_20231006_170130.jpg"
output_path = "/content/leaf_clean.png"

with open(input_path, "rb") as i:
    with open(output_path, "wb") as o:
        o.write(remove(i.read()))

Image.open(output_path).show()

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.2 MB/s eta 0:00:00


In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 🔹 Data preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/output/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/output/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# 🔹 Define CNN model inside Optuna
def create_model(trial):
    l2_reg = trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.7)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])

    model = Sequential([
        Conv2D(32, (3,3), padding='same', input_shape=(224,224,3), kernel_regularizer=l2(l2_reg)),
        Activation('relu'),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(l2_reg)),
        Activation('relu'),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), padding='same'),
        Activation('relu'),
        MaxPooling2D(2,2),

        Conv2D(256, (3,3), padding='same'),
        Activation('relu'),

        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(4, activation='softmax')
    ])

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def objective(trial):
    model = create_model(trial)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        callbacks=[early_stop],
        verbose=0
    )

    val_acc = max(history.history['val_accuracy'])
    return val_acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("Best trial:")
print("  Validation Accuracy: {:.4f}".format(study.best_value))
print("  Best Parameters:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")


Found 2022 images belonging to 4 classes.
Found 508 images belonging to 4 classes.


[I 2025-10-28 14:55:48,868] A new study created in memory with name: no-name-fd2a0b92-5fae-4f5d-8910-f827385bbca2
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
[I 2025-10-28 15:01:45,907] Trial 0 finished with value: 0.663385808467865 and parameters: {'l2_reg': 0.0004331620313549042, 'dropout_rate': 0.5456112579298483, 'learnin

KeyboardInterrupt: 